# Запись

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

drivers = [
    "/home/jovyan/work/spark-jars/hadoop-aws-3.3.4.jar",             # S3
    "/home/jovyan/work/spark-jars/aws-java-sdk-bundle-1.12.262.jar", # S3
    "/home/jovyan/work/spark-jars/wildfly-openssl-1.0.7.Final.jar",  # S3
    "/home/jovyan/work/spark-jars/postgresql-42.6.0.jar",            # PostgreSQL
]

spark = (SparkSession.builder
         .appName("mustdayker-Spark")
         .master("spark://spark-master:7077") 
         .config("spark.jars", ",".join(drivers))
         .getOrCreate()
        )

In [ ]:
spark.stop()

---

In [ ]:
data = [("Анна", 25), ("Иван", 30), ("Мария", 28), ("Осталый", 38)]
df = spark.createDataFrame(data, ["name", "age"])

In [ ]:
# Проверка записи в MinIO
df = spark.range(10)
df.write.mode("overwrite").csv("s3a://learn-bucket/test_22")
print("✅ Успешно!")

---

# PostgreSQL

In [ ]:
# Базовый

(df.write.format("jdbc")
         .option("url", "jdbc:postgresql://postgres-db:5432/learn_base")
         .option("driver", "org.postgresql.Driver")
         .option("user", "airflow")
         .option("password", "airflow")
         .option("dbtable", "отсталые")
         .mode("overwrite")
         .save())

In [ ]:
# Способы записи
         .mode("overwrite") # перезаписать таблицу
         .mode("append")    # добавить данные
         .mode("ignore")    # пропустить если таблица существует
         .mode("error")     # ошибка если таблица существует (по умолчанию)

In [ ]:
# Расширенный

(df.write.format("jdbc")
         .option("batchsize", 10000)                  # размер батча
         .option("isolationLevel", "READ_COMMITTED")  # уровень изоляции
         .option("truncate", "true")                  # truncate вместо drop при overwrite
         .option("createTableColumnTypes", "name VARCHAR(255), age INT") # типы колонок
)

In [ ]:
# Параллельная запись с партиционированием

(df.write.format("jdbc")
         .option("numPartitions", 10)         # количество партиций
         .option("partitionColumn", "id")     # колонка для партиционирования
         .option("lowerBound", "1")           # минимальное значение
         .option("upperBound", "1000000")     # максимальное значение
)

In [ ]:
# Через функцию

def spark_to_postgres(df, table_name, mode="append", **options):
    
    base_options = {
                    "url": "jdbc:postgresql://postgres-db:5432/learn_base",
                    "driver": "org.postgresql.Driver",
                    "user": "airflow", 
                    "password": "airflow",
                   }

    base_options.update(options)
    
    (df.write
       .format("jdbc")
       .option("dbtable", table_name)
       .options(**base_options)
       .mode(mode)
       .save())

# Использование
spark_to_postgres(df, "отсталые", "overwrite", batchsize=20000, truncate="true")

---

# CSV

In [ ]:
# Базовый
(df.write.format("csv")
    .option("header", "true")
    .option("delimiter", ",")
    .option("encoding", "UTF-8")
    .mode("overwrite")
    .save("s3a://my-bucket/data/csv-output"))

In [ ]:
# Полный
(df.write.format("csv")
    .option("header", "true")              # Заголовки
    .option("delimiter", ",")              # Разделитель
    .option("quote", "\"")                 # Символ quoting
    .option("escape", "\"")                # Символ экранирования
    .option("encoding", "UTF-8")           # Кодировка
    .option("nullValue", "NULL")           # Замена null
    .option("dateFormat", "yyyy-MM-dd")    # Формат дат
    .option("compression", "gzip")         # Сжатие
    .mode("overwrite")                     # Режим записи
    .save("s3a://my-bucket/data/output"))  # Путь назначения

# Parquet

In [ ]:
(df.write.format("parquet")
    .option("compression", "snappy")
    .option("parquet.block.size", 134217728)
    .option("parquet.page.size", 1048576)
    .option("parquet.dictionary.enabled", "true")
    .option("parquet.bloom.filter.enabled", "true")
    .mode("overwrite")
    .save("s3a://my-bucket/data/parquet-output"))

In [ ]:
(df.write.format("parquet")
    .option("compression", "snappy")
    .option("parquet.block.size", 134217728)
    .option("parquet.page.size", 1048576)
    .option("parquet.dictionary.enabled", "true")
    .option("parquet.bloom.filter.enabled", "true")
    .option("parquet.writer.version", "v2")
    .option("mergeSchema", "true")
    .partitionBy("year", "month")           # Партиционирование
    .mode("overwrite")
    .save("s3a://my-bucket/data/partitioned-parquet/"))

In [ ]:
spark.stop()